In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

# 缩写
tfd = tfp.distributions
tfpl = tfp.layers

# 创建示例数据
np.random.seed(0)
X = np.linspace(0, 1, 100).astype(np.float32)
Y = 1 + 2 * X + np.random.normal(0, 0.1, 100).astype(np.float32)

# 定义先验和后验分布
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential([
        tfpl.DistributionLambda(lambda t: tfd.MultivariateNormalDiag(loc=tf.zeros(n), scale_diag=tf.ones(n)))
    ])
    return prior_model

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = tf.keras.Sequential([
        tfpl.VariableLayer(n, dtype=dtype),
        tfpl.DistributionLambda(lambda t: tfd.MultivariateNormalDiag(loc=t, scale_diag=tf.nn.softplus(t)))
    ])
    return posterior_model

# 使用函数式 API 定义贝叶斯回归模型
inputs = tf.keras.Input(shape=(1,))
dense_variational = tfpl.DenseVariational(units=1, make_posterior_fn=posterior, make_prior_fn=prior, kl_weight=1/100)(inputs)
model = tf.keras.Model(inputs=inputs, outputs=dense_variational)

# 定义负对数似然损失函数
def negative_log_likelihood(y_true, y_pred):
    return -y_pred.log_prob(y_true)

# 编译模型
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=negative_log_likelihood)

# 训练模型
model.fit(X[:, np.newaxis], Y, epochs=1000, verbose=False)

# 预测
X_test = np.linspace(0, 1, 100).astype(np.float32)
Y_hat = model(X_test[:, np.newaxis])

# 绘制结果
plt.figure(figsize=(10, 6))
plt.scatter(X, Y, label='数据')
plt.plot(X_test, Y_hat.mean().numpy(), color='red', label='预测')
plt.fill_between(X_test, Y_hat.mean().numpy() - 2 * Y_hat.stddev().numpy(), Y_hat.mean().numpy() + 2 * Y_hat.stddev().numpy(), color='red', alpha=0.2, label='不确定性区间')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()


AttributeError: 'tuple' object has no attribute 'rank'